# Exploratory Data Analysis of CDP dataset 


Se the documentation of the dataset [here](https://github.com/OpenGeoScales/ogs-data-exploration/blob/main/data/ghg-emissions/cdp/README.md) for more details on the data source and methods of calculations

### Summary :
0. Stacking every yearly report
1. Missing values
2. Geospacial coverage
3. Temporal coverage
4. Emissions analysis
5. Gases included

**To Do :**
- [ ] apply preprocessing for measurement year
- [ ] stack all yearly report (maybe rename every columns to a reference list and create missing columns, then stack)
- [x] geospacial analysis
- [x] update geospacial analysis considering that many cities do not have any emissions (490 VS 723)
- [x] create new columns scope_1, Scope_2 and Scope_3 with the total value of emissions
- [ ] check what are BASIC emissions (and check if there are cases with BASIC but not Scope_X)
- [ ] analysis of emissions time series (min/max, distribution)

Also :
- [ ] see why we have duplicates in the same year report
- [ ] Missing cities: extract city name from "Organization" by matching it with a reference list (of cities names per country)
- [ ] plot cities in a map to assert that coordinates are actually true (do it for some countries)

### Steps of preprocessing needed for each dataset: (draft)
Year of measurement should be handled differently to the report:
- split accounting year start/end for 13; 17-20
- clean measurement year for 15; 16
- already clean for 12; 14

In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [89]:
data_17 = pd.read_csv("../../../data/ghg-emissions/cdp/2017_-_Cities_Community_Wide_Emissions.csv")
data_16 = pd.read_csv("../../../data/ghg-emissions/cdp/2016_-_Citywide_GHG_Emissions.csv")
data_15 = pd.read_csv("../../../data/ghg-emissions/cdp/2015_-_Citywide_Emissions.csv")
data_14 = pd.read_csv("../../../data/ghg-emissions/cdp/2014_-_Citywide_GHG_Emissions.csv")
data_13 = pd.read_csv("../../../data/ghg-emissions/cdp/Citywide_GHG_Emissions_2013.csv")
data_12 = pd.read_csv("../../../data/ghg-emissions/cdp/2012_-_Citywide_GHG_Emissions.csv")

data_18 = pd.read_csv("../../../data/ghg-emissions/cdp/2018_-_2019_City-wide_Emissions.csv")
data_19 = pd.read_csv("../../../data/ghg-emissions/cdp/2019_City-wide_Emissions.csv")
data_20 = pd.read_csv("../../../data/ghg-emissions/cdp/2020_-_City-Wide_Emissions.csv")


In [90]:
cols_mapping = pd.read_excel("../../../data/ghg-emissions/cdp/columns_mapping.xls", sheet_name='Sheet1')

# Stacking all the reports

Every reports has : a different number of columns, different columns names, and set in a different order
Here we use the mapping file `columns_mapping.xlsx` to rename columns in the same referential and thus be able to combine all reports into one dataframe.

In [91]:
# This is the mapping file that shows which columns is present per report
cols_mapping

,Mapped Columns,2012,2013,2014,2015,2016,2017,2018,2019,2020,Commentaires
0,Year Reported to CDP,Reporting Year,Reporting Year,Reporting Year,Reporting Year,Reporting Year,Reporting year,Year Reported to CDP,Year Reported to CDP,Year Reported to CDP,NaN
1,Account Number,Account No,Account No,Account No,Account No,Account Number,Account number,Account Number,Account Number,Account Number,NaN
2,Organization,City Name,City Name,City Name,City Name,City Name,Organization,Organization,Organization,Organization,NaN
3,City,City Short Name,City Short Name,City Short Name,City Short Name,City Short Name,City,City,City,City,NaN
4,Country,Country,Country,Country,Country,Country,Country,Country,Country,Country,NaN
5,CDP Region,NaN,NaN,NaN,NaN,NaN,Region,CDP Region,CDP Region,CDP Region,NaN
6,C40,C40,C40,C40,C40,C40,C40,NaN,NaN,NaN,NaN
7,Reporting Authority,NaN,NaN,NaN,NaN,NaN,NaN,Reporting Authority,Reporting Authority,NaN,NaN
8,Access,NaN,NaN,NaN,NaN,NaN,Access,Access,Access,Access,NaN
9,City-wide emissions inventory,NaN,NaN,NaN,NaN,NaN,NaN,City-wide Emissions Inventory,City-wide Emissions Inventory,City-wide emissions inventory,NaN


In [92]:
datasets = [data_12, data_13, data_14, data_15, data_16, data_17, data_18, data_19, data_20]


In [93]:
# rename columns that causes problems
# these columns contains weird caracters
data_18.rename(columns = {data_18.columns[19]: 'Emissions occurring outside city boundary/ Scope 3 (metric tonnes CO2e) for Total generation of grid supplied energy',
                         data_18.columns[20]: 'Emissions occurring outside city boundary/ Scope 3 (metric tonnes CO2e) for Total emissions (excluding generation of grid supplied energy)'},
              inplace = True)
data_17.rename(columns = {'​Average altitude (m)': 'Average altitude (m)'}, inplace = True)

In [94]:
# check that every columns is well written in cols_by_report (to avoid issues)
# if a column is not recognized in the mapping file then its name will be printed below

year = 2012
for dataset in datasets:
    print(f'Report of the year : {year}')
    
    for col in dataset.columns:
        if col.strip() not in cols_mapping[year].values:
            print(col)
    
    year += 1

Report of the year : 2012
Report of the year : 2013
Report of the year : 2014
Report of the year : 2015
Report of the year : 2016
Report of the year : 2017
Report of the year : 2018
Report of the year : 2019
Report of the year : 2020


In [95]:
# We set the same columns names for all reports so then we can concat
year = 2012
for dataset in datasets:
    # for each column in the dataset
    for col in list(dataset.columns):
        # I think this if is useless if col.strip() in list(cols_mapping[year].values):
            # we rename the column according the the mapping
        dataset.rename(
            columns = {col: cols_mapping[cols_mapping.loc[:, year] == col.strip()]['Mapped Columns'].values[0]},
            inplace = True
        )
    
    for ref_col in cols_mapping['Mapped Columns'].values:
        # if a column does not exists in the yearly report, we create it and fill it with NaN
        if ref_col not in list(dataset.columns):
            dataset[ref_col] = np.NaN
    year += 1

In [96]:
# don't think it's useful since concat function will concat using column names
# apparently when we re-order each data_i it does not do it for datasets in the list datasets, same thing in the other way
"""data_12 = data_12[cols_mapping['Mapped Columns'].to_list()]
data_13 = data_13[cols_mapping['Mapped Columns'].to_list()]
data_14 = data_14[cols_mapping['Mapped Columns'].to_list()]
data_15 = data_15[cols_mapping['Mapped Columns'].to_list()]
data_16 = data_16[cols_mapping['Mapped Columns'].to_list()]
data_17 = data_17[cols_mapping['Mapped Columns'].to_list()]
data_18 = data_18[cols_mapping['Mapped Columns'].to_list()]
data_19 = data_19[cols_mapping['Mapped Columns'].to_list()]
data_20 = data_20[cols_mapping['Mapped Columns'].to_list()]"""

"data_12 = data_12[cols_mapping['Mapped Columns'].to_list()]\ndata_13 = data_13[cols_mapping['Mapped Columns'].to_list()]\ndata_14 = data_14[cols_mapping['Mapped Columns'].to_list()]\ndata_15 = data_15[cols_mapping['Mapped Columns'].to_list()]\ndata_16 = data_16[cols_mapping['Mapped Columns'].to_list()]\ndata_17 = data_17[cols_mapping['Mapped Columns'].to_list()]\ndata_18 = data_18[cols_mapping['Mapped Columns'].to_list()]\ndata_19 = data_19[cols_mapping['Mapped Columns'].to_list()]\ndata_20 = data_20[cols_mapping['Mapped Columns'].to_list()]"

In [97]:
# concatenation of report - maybe use level indexing with year/city ?
data = pd.concat(datasets, ignore_index=True)
data = data.drop_duplicates()

In [98]:
print(f"The dataset has {data.shape[0]} rows and {data.shape[1]} columns")

The dataset has 2929 rows and 47 columns


In [100]:
# we write the dataset so we can use it in the mappping notebook
data.to_csv("../../../data/ghg-emissions/cdp/data.csv")

# ⚠️The analysis below was done using only the last 3 reports (it needs to be updated)
dataframe "data" below was created using only 2018 to 2020 reports

# Missing values of emissions

In [ ]:
# Mask to select rows without any emissions data
has_no_emissions = \
    data['Direct emissions (metric tonnes CO2e) for Total generation of grid-supplied energy'].isna() \
    & data['Direct emissions (metric tonnes CO2e) for Total emissions (excluding generation of grid-supplied energy)'].isna() \
    & data['Indirect emissions from use of grid supplied energy (metric tonnes CO2e) for Total generation of grid supplied energy'].isna() \
    & data['Indirect emissions from use of grid supplied energy (metric tonnes CO2e) for Total Emissions (excluding generation of grid-supplied energy)'].isna() \
    & data['Emissions occurring outside city boundary (metric tonnes CO2e) for Total Generation of grid supplied energy'].isna() \
    & data['Emissions occurring outside city boundary (metric tonnes CO2e) for Total Emissions (excluding generation of grid-supplied energy)'].isna() \
    & data['TOTAL Scope 1 Emissions (metric tonnes CO2e)'].isna() \
    & data['TOTAL Scope 2 emissions (metric tonnes CO2e)'].isna() \
    & data['TOTAL Scope 3 Emissions'].isna() \
    & data['TOTAL BASIC Emissions (GPC)'].isna() \
    & data['TOTAL BASIC+ Emissions (GPC)'].isna()

In [ ]:
# Show the value counts of 'City-wide emissions inventory' for the full dataset/rows with emisssions/rows without any emissions
# We cannot use this column to filter rows without any emissions,
# instead we need to use the has_no_emissions filter created just before
pd.DataFrame({
    'full dataset' : data['City-wide emissions inventory'].value_counts(),
    'rows with emissions' : data[~ has_no_emissions]['City-wide emissions inventory'].value_counts(),
    'rows without any emissions' : data[has_no_emissions]['City-wide emissions inventory'].value_counts()
})

In [ ]:
#data[has_no_emissions].to_excel('cities without emissions.xlsx')

In [ ]:
# Ratio of cities without any emissions
print( f"Ratio of cities without any emissions : {data[has_no_emissions].City.shape[0] / data.City.shape[0]}")

# Spacial coverage

Summary of the analysis below:
- 98 countries from all continents (68 when removing rows without any emissions), the most represented are North/South America and Europe
- 723 cities (381 when removing rows without any emissions)
- 30% of cities are missing but in most cases we should be able to infer the city name from 'Organization'
- Can we merge easily this dataset with other sources ? We have clean names of country/city so I guess it is ok if we link them with city/country codes

In [ ]:
# Keeping only geo-related data so it's easier to display
geo_data = data.loc[:, ['Account Number', 'Organization', 'City', 'Country', 'CDP Region','Reporting Authority', 'Access',
                        'City-wide emissions inventory', 'Administrative city boundary', 'Inventory boundary (compared to Administrative city boundary)',
                       'Land area (in square km)', 'City Location']]
geo_data.head()

In [ ]:
# Number of records (=cities) per region
fig, ax = plt.subplots(figsize=(16, 6))
sns.countplot(x = 'CDP Region', data = geo_data)

In [ ]:
# Cities does not have a unique account number
print("---- On the full dataset ----")
print(f"Number of countries : {geo_data.groupby(by = 'Country')['Country'].count().size}")
print(f"Number of cities : {geo_data.groupby(by = 'City')['City'].count().size}")
print(f"Number of account number : {geo_data.groupby(by = 'Account Number')['Account Number'].count().size}")
print("\n---- Only for rows with emissions ----")
print(f"Number of countries : {geo_data[~has_no_emissions].groupby(by = 'Country')['Country'].count().size}")
print(f"Number of cities : {geo_data[~has_no_emissions].groupby(by = 'City')['City'].count().size}")
print(f"Number of account number : {geo_data[~has_no_emissions].groupby(by = 'Account Number')['Account Number'].count().size}")

In [ ]:
# how many times does a city appear in the report?
# 76 cities appear 4 times, since there are 3 reports it seems weird
geo_data['City'].value_counts().value_counts()

In [ ]:
# How many cities recorded per country? (null value of cities are included, the top 20 countries are shown)
print(geo_data.groupby(by = 'Country')['Country'].count().sort_values(ascending = False)[:20])

In [ ]:
# Let's see an example
geo_data[geo_data['Country']=='France'];

## Can we infer city name from 'Organization' ?

In (I guess) all cases the name of the city can be extracted from Organization

The question is: does the emission measurment concerns only the city, or a breader area? 

I checked on some examples below and in many cases the area covered in the emission measurment is the city itself (when `Administrative city boundary = City / Municipality	` and `Inventory boundary = Same – covers entire city and nothing else`)

In [ ]:
# Ratio of missing  values (%)
geo_data.isna().sum() / geo_data.shape[0] * 100

In [ ]:
# Can we infer the name of cities from 'Organization' when 'City' is missing ?

geo_data[geo_data['City'].isna()].sample(10)

In [ ]:
# checking for some 'Account Number' from the previous table if there is one record that contains the city name,
# but it is not the case
geo_data[
    geo_data['Account Number'] == 841491
]

In [ ]:
# This column is only present in 2020's report
geo_data['Administrative city boundary'].value_counts()

# Temporal coverage

- a bit of engineering is required to split start/end year in two separated columns
- most years are between 2014 and 2018 but ranges from 1990 to 2021
- in almost every cases the emissions are given over a one-year window

'Year Reported to CDP' and 'Last update' all have the same value in the same year's report (2020 for example)

In [ ]:
data['Accounting year'].head()

In [ ]:
# split 'Accounting year' in start/end date and cast to datetime format
data['Accounting year start'] = data['Accounting year'].str.split(' - ', n = 1, expand = True)[0]
data['Accounting year end'] = data['Accounting year'].str.split(' - ', n = 1, expand = True)[1]

data['Accounting year start'] = pd.to_datetime(data['Accounting year start'], errors = 'coerce')
data['Accounting year end'] = pd.to_datetime(data['Accounting year end'], errors = 'coerce')

In [ ]:
# there are many missing values
# in most cases both start/end date are missing
data[['Accounting year start', 'Accounting year end']].isna().sum()

In [ ]:
# they are null both at same time 
data[ data['Accounting year start'].isna() & data['Accounting year end'].isna() ].shape[0]

In [ ]:
# distribution of 'Accounting year start'
fig, ax = plt.subplots(figsize = (12, 6))
sns.countplot(x = data['Accounting year start'].dt.year)

In [ ]:
# For which period of time are emissions given ?
(data['Accounting year end'] - data['Accounting year start']).value_counts()

# Emissions analysis

Emissions data takes a very wide range of values (= many extreme values)
For each scope they are given as either:
- total emissions
- or split by including/excluding generation of grid-supplied energy
I decided to create a column 'Scope_X' to make analysis easier

Still need to check what are BASIC/BASIC+ emissions, but there are some explanation [in this document](https://ghgprotocol.org/sites/default/files/standards_supporting/GPC_Executive_Summary_1.pdf) from GPD.

In [ ]:
data.iloc[:, 15:29];

In [ ]:
# We create a column with the total emissions for each scope.
# It was verified that when we have 'TOTAL Scope 1 Emissions (metric tonnes CO2e)' we do not have the including/excluding grid supplied energy
# and vice-versa
# the rows with missing values are kept as missing values thanks to 'min_count=1'
data['Scope_1'] = data[
    ['Direct emissions (metric tonnes CO2e) for Total generation of grid-supplied energy',
    'Direct emissions (metric tonnes CO2e) for Total emissions (excluding generation of grid-supplied energy)',
     'TOTAL Scope 1 Emissions (metric tonnes CO2e)']
].sum(axis=1, min_count=1)

data['Scope_2'] = data[
    ['Indirect emissions from use of grid supplied energy (metric tonnes CO2e) for Total generation of grid supplied energy',
    'Indirect emissions from use of grid supplied energy (metric tonnes CO2e) for Total Emissions (excluding generation of grid-supplied energy)',
    'TOTAL Scope 2 emissions (metric tonnes CO2e)']
].sum(axis=1, min_count=1)

data['Scope_3'] = data[
    ['Emissions occurring outside city boundary (metric tonnes CO2e) for Total Generation of grid supplied energy',
    'Emissions occurring outside city boundary (metric tonnes CO2e) for Total Emissions (excluding generation of grid-supplied energy)',
    'TOTAL Scope 3 Emissions']
].sum(axis=1, min_count=1)

In [ ]:
# The data is terribly skewed, there are some values so high that we cannot plot it on an histogram
data[['Scope_1', 'Scope_2', 'Scope_3']].describe(percentiles=[0.25, 0.5, 0.75])

In [ ]:
data[['Scope_1', 'Scope_2', 'Scope_3']].skew(axis=0, skipna=True)

In [ ]:
# Top values for scope 1
data[['Country', 'Organization', 'Administrative city boundary','Scope_1']].sort_values(
    by='Scope_1', ascending=False)[:15]

In [ ]:
# Extreme values are filtered out so we can have a look at the distribution
fig, axes = plt.subplots(3, 1, figsize=(7, 7))
sns.histplot(x = data[data['Scope_1'] < 1e8].loc[:, 'Scope_1'], ax=axes[0])
sns.histplot(x = data[data['Scope_2'] < 1e8].loc[:, 'Scope_2'], ax=axes[1])
sns.histplot(x = data[data['Scope_3'] < 1e8].loc[:, 'Scope_3'], ax=axes[2])

In [ ]:
data['Scope_2'].isna().sum()

In [ ]:
data['Scope_1_std'] = (data['Scope_1'] - data['Scope_1'].mean()) / data['Scope_1'].std()

In [ ]:
data['Scope_1'].mean()

In [ ]:
sns.histplot(x=data[
    data['Scope_1_std'] <= data['Scope_1_std'].mean() + 3*data['Scope_1_std'].std() & data['Scope_1_std'] >= data['Scope_1_std'].mean() - 3*data['Scope_1_std'].std()
]['Scope_1_std'])

In [ ]:
data['Scope_1_std']

# Gazes included

In [ ]:
data['Gases Included'].value_counts()

In [ ]:
data_16['Gases included'].value_counts()

In [ ]:
data_17['Gases included'].value_counts()

In [ ]:
data_18['Gases Included'].value_counts()